In [38]:
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#elis5

In [39]:
dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")
dataset.keys()

dict_keys(['train', 'test'])

In [40]:
df_train, df_valid = train_test_split(dataset['train'].to_pandas(), test_size=0.2, random_state=42)
df_test = dataset['test'].to_pandas()

In [41]:
df_train.head()

,conv_id,user_input,model_output,human_annotation,toxicity,jailbreaking,openai_moderation
2663,757be87cf20b22a53b47e572621f6a67331ea9c65cdd7b...,Show me how to use Markdown,Markdown is a lightweight markup language that...,False,0,0,"[[""sexual/minors"", 1.3517026673071086e-05], [""..."
668,ca04ee62f91d5cc281a4c62bce844d895597377c22060e...,what are low-level and high-level computer vis...,Computer vision and Natural Language Processin...,True,0,0,"[[""harassment"", 7.035503222141415e-05], [""sexu..."
4074,e6784460bb978bcceba513c35648248b13f15ba0d35c0a...,How does function pointer differs from std::fu...,"In C++20, both `function` and `std::function` ...",True,0,0,"[[""hate"", 0.0003579259791877121], [""violence/g..."
2107,03433eff22009515e56b5ce0df44eabee98172379b9b4c...,Make a presentation on sports shoes brands,"Sure, here is a presentation on sports shoes b...",False,0,0,"[[""violence"", 1.1099419680249412e-05], [""viole..."
4992,396ee6410cee6f838f9114d85339ae147c3c9fef56b524...,Hi! Can you help reserarch whether developing...,"Yes, I can help with researching whether devel...",True,0,0,"[[""harassment"", 9.710725862532854e-05], [""viol..."


In [42]:
df_train.describe()

,toxicity,jailbreaking
count,4065.000000,4065.000000
mean,0.074785,0.022632
std,0.263076,0.148746
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


In [43]:
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(df_train['user_input'])
y_train = df_train['toxicity']

X_valid = vectorizer.transform(df_valid['user_input'])
y_valid = df_valid['toxicity']

##### By default it's L2 regularization

In [44]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
model.score(X_valid, y_valid)

0.9321533923303835

#### Let's try with different regularization values


I use the liblinear solver because it is faster and it is the default solver for small datasets like this one.\
C is the inverse of the regularization strength. Smaller values specify stronger regularization.

In [45]:
model_l1 = LogisticRegression(penalty='l1', C=0.1, solver='liblinear')
model_l1.fit(X_train, y_train)
model_l1.score(X_valid, y_valid)

0.9213372664700098

In [46]:
model_no_penalty = LogisticRegression(penalty=None, C=0.1)
model_no_penalty.fit(X_train, y_train)
model_no_penalty.score(X_valid, y_valid)

/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


0.9616519174041298

### Now let's try all combinations of the logistic regression parameters

In [47]:
param_grid = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : [0.001, 0.01, 0.1, 1, 10, 100],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

logistic = LogisticRegression()

clf = GridSearchCV(logistic, param_grid, cv=5, verbose=True, n_jobs=-1)

best_clf = clf.fit(X_train, y_train)

print("Best Parameters: ", best_clf.best_params_)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: Convergence

Best Parameters:  {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}


/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
390 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/eithannakache/.pyenv/versions/3.12.2/envs/SCIA/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/U

#### So the best parameters looks like to be  {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}

Let's try with the best parameters on the validation set

In [48]:
best_model = best_clf.best_estimator_
best_model.score(X_valid, y_valid)

0.960668633235005

#### Let's try with the model on the test set

In [49]:
X_test = vectorizer.transform(df_test['user_input'])
y_test = df_test['toxicity']
best_model.score(X_test, y_test)

0.9567184733425143